In [9]:
%run CompilerTheory_Ch5(Tree_Classes).ipynb

In [10]:
%run Lexical_Analyzer.ipynb

In [11]:
%run parser_helpers.ipynb

In [12]:
%run Translator.ipynb

In [13]:
%run Intermediate_Code_Generator.ipynb

In [14]:
%run Code_Generator.ipynb

In [15]:
def Compile(path,v=0,vv=0):
    """compiles the source m"""
    
    global ST         # symbol table        
    target = ""       # string for target code
    
    
    file_ptr = open(path,"r")                  # open file for read mode
    source = file_ptr.read()                   # read file to string m
    file_ptr.close()                           # close file after reading
    if v: print("source:\n"+source+"\n\n")
    
    
    
    #-------------------Compiler Front-End---------------------------------------------------------
    Parse_Tree = parser(source)                               # call the parser/translator
    Processed_Tree = pre_process(Parse_Tree)                  # pre-process the tree
    if vv: printTree(Processed_Tree,1,2000,14)
    
    
    
    #-------------------Compiler Back-End----------------------------------------------------------
    Quadruples = int_code_gen(Processed_Tree,v,vv)            # call the code generator
    
    for Q in Quadruples:
        if vv: print(Q)
        target += code_gen(Q)
    
    
    fp = open(path[:-2]+".asm",'w')            # open file for output
    
    
    fp.write("extrn ExitProcess : proc\n\n")   # file header
    fp.write(".data\n")                        # .data section
    
    for key in ST:                             # declare variables (in MASM)
        if key not in ["", 'int', 'float', 'if', 'else', 'for', 'while', 'do', 'break', 'continue', 'goto']:
            if ST[key].array_size:                 # check if the variable is an array
                fp.write(key+" QWORD "+str(ST[key].array_size)+" DUP (?)\n")    # initialize an empty array
            else:
                fp.write(key+" QWORD ?\n")
    
    fp.write("\n\n.code\n")                    # .code section
    fp.write("_main PROC\n")
    
    
    fp.write(target)                           # code generator output
    
    
    trailer = "\n\n\ndone:\nxor rcx, rcx\ncall ExitProcess\n_main ENDP\nEND" 
    fp.write(trailer)                          # file trailer
    fp.close()                                 # close output file
    print("compilation complete")

In [17]:
Compile("tester.c")


Parsing Complete
compilation complete
